In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from templates import *
from templates_cls import *
from experiment_classifier import ClsModel

In [3]:
device = 'cuda:0'
conf = ffhq256_autoenc()
# print(conf.name)
model = LitModel(conf)
state = torch.load(f'checkpoints/{conf.name}/last.ckpt', map_location='cpu')
model.load_state_dict(state['state_dict'], strict=False)
model.ema_model.eval()
model.ema_model.to(device);

Global seed set to 0


Model params: 160.69 M


In [4]:
cls_conf = ffhq256_autoenc_cls()
cls_model = ClsModel(cls_conf)
state = torch.load(f'checkpoints/{cls_conf.name}/last.ckpt',
                    map_location='cpu')
print('latent step:', state['global_step'])
cls_model.load_state_dict(state['state_dict'], strict=False)
cls_model.to(device);

Global seed set to 0


loading pretrain ... 130M
step: 1563562
loading latent stats ...
latent step: 9375


## 選択

In [5]:
img_pose_dir = 'videos_test/video1/pose/'
img_eye_dir = 'videos_test/video1/syn/eye/'
img_mouth_dir = 'videos_test/video1/syn/mouth/'
img_to_dir = 'videos_test/img/'
for i,file in enumerate(os.listdir(img_to_dir)):
    print(f'{i:2}: {file}')

 0: frame_001.png


In [6]:
data_pose = ImageDataset(img_pose_dir, image_size=conf.img_size, exts=['jpg', 'JPG', 'png'], do_augment=False, sort_names=True)
data_eye = ImageDataset(img_eye_dir, image_size=conf.img_size, exts=['jpg', 'JPG', 'png'], do_augment=False, sort_names=True)
data_mouth = ImageDataset(img_mouth_dir, image_size=conf.img_size, exts=['jpg', 'JPG', 'png'], do_augment=False, sort_names=True)
data_to = ImageDataset(img_to_dir, image_size=conf.img_size, exts=['jpg', 'JPG', 'png'], do_augment=False, sort_names=True)
batch_pose = torch.tensor([])
batch_eye = torch.tensor([])
batch_mouth = torch.tensor([])
batch_to = torch.tensor([])
names_batch_to = []
for i in range(len(data_pose)):
    batch_pose = torch.cat([batch_pose, data_pose[i]['img'][None]], axis=0)
    batch_eye = torch.cat([batch_eye, data_eye[i]['img'][None]], axis=0)
    batch_mouth = torch.cat([batch_mouth, data_mouth[i]['img'][None]], axis=0)

for i in range(len(data_to)):
    batch_to = torch.cat([batch_to, data_to[i]['img'][None]], axis=0)
    names_batch_to.append(os.path.splitext(str(data_to.paths[i]))[0])

In [7]:
cond_pose = torch.tensor([], device=device)
cond_eye = torch.tensor([], device=device)
cond_mouth = torch.tensor([], device=device)
cond_to = torch.tensor([], device=device)
cond_to_norm = torch.tensor([], device=device)
xT_to = torch.tensor([], device=device)

for i in range(len(batch_pose)):
    cond_pose = torch.cat([cond_pose, model.encode(batch_pose[i][None].to(device))], dim=0)
    cond_eye = torch.cat([cond_eye, model.encode(batch_eye[i][None].to(device))], dim=0)
    cond_mouth = torch.cat([cond_mouth, model.encode(batch_mouth[i][None].to(device))], dim=0)

for i in range(len(batch_to)):
    cond_to = torch.cat([cond_to, model.encode(batch_to[i][None].to(device))], dim=0)
    cond_to_norm = torch.cat([cond_to_norm, cls_model.normalize(cond_to[i])], dim=0)
    xT_to = torch.cat([xT_to, model.encode_stochastic(batch_to[i][None].to(device), cond_to[i][None], T=250)], dim=0)

## pose

In [8]:
cond_change_pose = torch.tensor([], device=device)
for i in range(len(batch_pose)):
    cond_change = cls_model.normalize(cond_pose[i][None]) - cls_model.normalize(cond_pose[0][None])
    cond_change_pose = torch.cat([cond_change_pose, cond_change], dim=0)

## eye

In [9]:
cond_change_eye = torch.tensor([], device=device)
for i in range(len(batch_eye)):
    cond_change = cls_model.normalize(cond_eye[i][None]) - cls_model.normalize(cond_eye[0][None])
    cond_change_eye = torch.cat([cond_change_eye, cond_change], dim=0)

## mouth

In [10]:
cond_change_mouth = torch.tensor([], device=device)
for i in range(len(batch_mouth)):
    cond_change = cls_model.normalize(cond_mouth[i][None]) - cls_model.normalize(cond_mouth[0][None])
    cond_change_mouth = torch.cat([cond_change_mouth, cond_change], dim=0)

## 推論

In [11]:
%%time
preds = torch.tensor([], device=device)
for i in range(len(batch_to)):
    preds_tmp = torch.tensor([], device=device)
    for j in range(len(batch_pose)):
        cond2 = cond_to_norm[i] + cond_change_pose[j]
        cond2 = cls_model.denormalize(cond2)
        pred = model.render(xT_to[i][None], cond2, T=20)
        preds_tmp = torch.cat([preds_tmp, pred], dim=0)
    preds = torch.cat([preds, preds_tmp[None]], dim=0)

CPU times: total: 12min 7s
Wall time: 16min 35s


In [12]:
from PIL import Image

preds_pil = []
for pred in preds:
    pred_pil = []
    for p in pred:
        p_np = np.array((p*255).permute(1,2,0).cpu()).astype(np.uint8)
        pred_pil.append(Image.fromarray(p_np))
    preds_pil.append(pred_pil)

## gifの保存

In [13]:
dst_dir = f"videos_test/results/"

In [14]:
for i,pred_pil in enumerate(preds_pil):
    dst_dir_gif = f"{dst_dir}/gif/"
    if not os.path.exists(dst_dir_gif): os.makedirs(dst_dir_gif)
    file_name = f"result_pose.gif"
    dst_path = dst_dir_gif + file_name

    pred_pil[0].save(
        dst_path,
        format="gif",
        save_all=True,
        append_images=pred_pil,
        duration=33,
        loop=0,
    )

## 静止画像（フレーム）の保存

In [130]:
from torchvision.utils import save_image

select_ind = set(range(len(batch_pose))
for i,pred in enumerate(preds):
    dst_dir_frames = f"{dst_dir}/frames/"
    if not os.path.exists(dst_dir_frames): os.makedirs(dst_dir_frames)
    for j,p in enumerate(pred):
        if j in select_ind:
            file_name = f"{dst_dir_frames}{j+1:03}.png"
            save_image(p, file_name, format='PNG')